# Introduction

This notebook shows a way of predicting House Prices using the advantages of Linear regression and XGBoost. Firstly, the main information in both train and test data was cleaned alongside a simple Feature Engineering to fill the missing values.

After, a simple Linear Regression of the significant values through the P-value criteria was chosen ($P_{value}<0.05$) alongside the elimination of columns that caused problems in the final prediction.

Then, it was made a Grid Search to find the best parameters of the XGBoost modeling with the train data related to the maximum depth and number of iterators. With those data, it was constructed a pipeline with the `XGBoostRegressor` and a `StandardScaler`.

Finally, the results with the test data were saved on the file `Submission.csv`.

# 1. Libraries and test-train data

In [1]:
#Numerical libraries
import numpy as np
import pandas as pd
#Tool of statsmodel for using the OLS method of Linear Regression
import statsmodels.formula.api as smf
#Creation of overall pipeline
from sklearn.pipeline import make_pipeline
#Standard scaler
from sklearn.preprocessing import StandardScaler
#Main regressor
from xgboost import XGBRegressor
#GridSearch tool
from sklearn.model_selection import GridSearchCV

With those libraries, the main data was extracted. In this case, both Train and Test columns have NaN values, which in the `data_description.txt` file states that in some categories this is normal as it is specified that there is not that specific property on the houses like Pools and Alley.

In [2]:
train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


With this in consideration, 14 columns were chosen to replace the NaN values with the text "None", which can be used by the regression modeling and that has the specific category of none.

In [4]:
#List of columns with NaN data - category
special_cols=["Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1",
              "BsmtFinType2","FireplaceQu","GarageType","GarageFinish",
              "GarageQual","GarageCond","PoolQC","Fence","MiscFeature"]
for i in special_cols: #Train data
    null_cols=train[train[i].isna()].index
    for n in null_cols:
        train.loc[n,i]="None"
for i in special_cols: #Test data
    null_cols=test[test[i].isna()].index
    for n in null_cols:
        test.loc[n,i]="None"

After, the columns of `LotFrontage`, `GarageYrBlt`, `MasVnrType` and `MasVnrArea` were replaced to continue with the filling of missing elements. In this case for `LotFrontage` and `MasVnrArea` the missing data was replaced with 0, `GarageYrBlt` with the mode and `MasVnrType` with "None".

In the case of `Electrical` from test data, the missing values were replaced by the mode.

In [5]:
#Train data
#Filling LotFrontage
null_cols=train[train["LotFrontage"].isna()].index
for n in null_cols:
    train.loc[n,"LotFrontage"]=0
#Filling GarageYrBLt
null_cols=train[train["GarageYrBlt"].isna()].index
for n in null_cols:
    train.loc[n,"GarageYrBlt"]=train["GarageYrBlt"].mode()[0]
#Filling MasVnrType
null_cols=train[train["MasVnrType"].isna()].index
for n in null_cols:
    train.loc[n,"MasVnrType"]="None"
#Filling MasVnrArea
null_cols=train[train["MasVnrArea"].isna()].index
for n in null_cols:
    train.loc[n,"MasVnrArea"]=0

In [6]:
#Test data
#Filling LotFrontage
null_cols=test[test["LotFrontage"].isna()].index
for n in null_cols:
    test.loc[n,"LotFrontage"]=0
#Filling GarageYrBLt
null_cols=test[test["GarageYrBlt"].isna()].index
for n in null_cols:
    test.loc[n,"GarageYrBlt"]=test["GarageYrBlt"].mode()[0]
#Filling MasVnrType
null_cols=test[test["MasVnrType"].isna()].index
for n in null_cols:
    test.loc[n,"MasVnrType"]="None"
#Filling MasVnrArea
null_cols=test[test["MasVnrArea"].isna()].index
for n in null_cols:
    test.loc[n,"MasVnrArea"]=0
#Filling Electrical
null_cols=train[train["Electrical"].isna()].index
for n in null_cols:
    train.loc[n,"Electrical"]=train["Electrical"].mode()[0]

Then, for additional verification for the remaining cells with a negative value the following cell shows the filling of the test dataset in two ways:

1. Object -> replace with the mode
2. Numerical -> replace with the column mean

In [7]:
Rem_null_test=[]
for i in test.columns:
    if test[i].isna().value_counts()[0]-len(test[i])<0:
        Rem_null_test.append(i)

In [8]:
for col in Rem_null_test:
    if test[col].dtype == 'O': #If the column is of dtype object
        null_cols=test[test[col].isna()].index
        for n in null_cols:
            test.loc[n,col]=test[col].mode()[0]
    else: #Then in this case is numerical
        null_cols=test[test[col].isna()].index
        for n in null_cols:
            test.loc[n,col]=test[col].mean()

Finally in this section, it is shown a code for checking if all the columns in the dataset don't have missing values for continuing the Linear Regression and XGBoost analysis

In [9]:
#Checking if there are columns with complete values in train dataset
yes_null_cols_train=[]
for i in test.columns:
    if train[i].isna().value_counts()[0]-len(train[i])<0:
        yes_null_cols_train.append(i)
len(yes_null_cols_train)

0

In [10]:
#Checking if there are columns with complete values in test dataset
yes_null_cols_test=[]
for i in test.columns:
    if test[i].isna().value_counts()[0]-len(test[i])<0:
        yes_null_cols_test.append(i)
len(yes_null_cols_test)

0

# 2. Linear regression

With all the data cleaned and filled, in the next cell, the main formula concerning the Linear regression is built. In this case, every column of the test dataset is included and the main objective was to find which values are statistically significant.

However, to build this function it was used the pasty code `Q()` for avoiding problems in function building and with columns with numbers in their name.

In [11]:
formula ="SalePrice ~ "
columns = ""
for i in train.columns[1:-1]:
        if columns == "":
            columns = columns + "Q('" + str(i) + "')"
        else:
            columns = columns + " + Q('" + str(i) + "')"
columns

"Q('MSSubClass') + Q('MSZoning') + Q('LotFrontage') + Q('LotArea') + Q('Street') + Q('Alley') + Q('LotShape') + Q('LandContour') + Q('Utilities') + Q('LotConfig') + Q('LandSlope') + Q('Neighborhood') + Q('Condition1') + Q('Condition2') + Q('BldgType') + Q('HouseStyle') + Q('OverallQual') + Q('OverallCond') + Q('YearBuilt') + Q('YearRemodAdd') + Q('RoofStyle') + Q('RoofMatl') + Q('Exterior1st') + Q('Exterior2nd') + Q('MasVnrType') + Q('MasVnrArea') + Q('ExterQual') + Q('ExterCond') + Q('Foundation') + Q('BsmtQual') + Q('BsmtCond') + Q('BsmtExposure') + Q('BsmtFinType1') + Q('BsmtFinSF1') + Q('BsmtFinType2') + Q('BsmtFinSF2') + Q('BsmtUnfSF') + Q('TotalBsmtSF') + Q('Heating') + Q('HeatingQC') + Q('CentralAir') + Q('Electrical') + Q('1stFlrSF') + Q('2ndFlrSF') + Q('LowQualFinSF') + Q('GrLivArea') + Q('BsmtFullBath') + Q('BsmtHalfBath') + Q('FullBath') + Q('HalfBath') + Q('BedroomAbvGr') + Q('KitchenAbvGr') + Q('KitchenQual') + Q('TotRmsAbvGrd') + Q('Functional') + Q('Fireplaces') + Q('Fir

In [12]:
final_formula = formula + columns
model = smf.ols(formula=final_formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     67.22
Date:                Mon, 08 Aug 2022   Prob (F-statistic):               0.00
Time:                        01:01:11   Log-Likelihood:                -16569.
No. Observations:                1460   AIC:                         3.364e+04
Df Residuals:                    1208   BIC:                         3.497e+04
Df Model:                         251                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -5.861e+05   1.06e+06     -0.554      0.579   -2.66e+06    1.49e+06
Q('MSZoning')[T.FV]            3.237e+04    1.2e+04      2.700      0.007    8852.847    5.59e+04
Q('MSZoning')[T.RH]             2.25e+04   1.19e+04      1.894      0.058    -802.966    4.58e+04
Q('MSZoning')[T.RL]            2.502e+04   1.02e+04      2.451      0.014    4989.877    4.51e+04
Q('MSZoning')[T.RM]            2.171e+04   9575.100      2.267      0.024    2920.793    4.05e+04
Q('Street')[T.Pave]            3.327e+04   1.22e+04      2.733      0.006    9382.756    5.72e+04
Q('Alley')[T.None]            -1310.0558   4206.658     -0.311      0.756   -9563.223    6943.111
Q('Alley')[T.Pave]             -832.6495   6010.240     -0.139      0.890   -1.26e+04     1.1e+04
Q('LotShape')[T.IR2]           4992.8132   4207.957      1.187      0.236   -3262.903    1.32e+04
Q('LotShape')[T.IR3]           6257.3404   8799.801      0.711      0.477    -1.1e+04    2.35e+04
Q('LotShape')[T.Reg]           1727.4752   1633.333      1.058      0.290   -1477.009    4931.960
Q('LandContour')[T.HLS]        7563.9532   5122.185      1.477      0.140   -2485.413    1.76e+04
Q('LandContour')[T.Low]       -1.131e+04   6396.864     -1.768      0.077   -2.39e+04    1238.288
Q('LandContour')[T.Lvl]        5413.9874   3698.787      1.464      0.144   -1842.772    1.27e+04
Q('Utilities')[T.NoSeWa]      -3.761e+04   2.63e+04     -1.430      0.153   -8.92e+04     1.4e+04
Q('LotConfig')[T.CulDSac]      7933.5304   3244.916      2.445      0.015    1567.234    1.43e+04
Q('LotConfig')[T.FR2]         -7738.1831   3994.817     -1.937      0.053   -1.56e+04      99.368
Q('LotConfig')[T.FR3]         -1.705e+04   1.25e+04     -1.360      0.174   -4.17e+04    7545.131
Q('LotConfig')[T.Inside]      -1524.3656   1753.147     -0.870      0.385   -4963.916    1915.185
Q('LandSlope')[T.Mod]          7565.8489   3966.438      1.907      0.057    -216.024    1.53e+04
Q('LandSlope')[T.Sev]         -4.198e+04   1.14e+04     -3.691      0.000   -6.43e+04   -1.97e+04
Q('Neighborhood')[T.Blueste]   7097.8066   1.92e+04      0.370      0.712   -3.06e+04    4.48e+04
Q('Neighborhood')[T.BrDale]   -2805.2574    1.1e+04     -0.256      0.798   -2.43e+04    1.87e+04
Q('Neighborhood')[T.BrkSide]  -5825.6600   9471.304     -0.615      0.539   -2.44e+04    1.28e+04
Q('Neighborhood')[T.ClearCr]  -1.456e+04   9193.907     -1.584      0.113   -3.26e+04    3475.782
Q('Neighborhood')[T.CollgCr]  -1.015e+04   7246.634     -1.401      0.162   -2.44e+04    4066.391
Q('Neighborhood')[T.Crawfor]    1.17e+04   8534.547      1.370      0.171   -5048.483    2.84e+04
Q('Neighborhood')[T.Edwards]   -2.16e+04   7987.826     -2.704      0.007   -3.73e+04   -5925.631
Q('Neighborhood')[T.Gilbert]  -1.127e+04   76

With those results, the not significant columns are erased from the Linear Regression model to see if the R-squared with fewer variables keeps almost the same value. For choosing those variables the following criteria were applied:

1. If in categorical columns at least one category has a P-value of less than 0.5 the overall variable is preserved
2. In a numerical column only a variable is preserved if the P-value is less than 0.5

After the analysis, the `model2` was trained:

In [13]:
#Eliminating not significant columns (Pvalue>0.05)
not_sig_cols=["Alley", "LotShape", "LandContour", "Utilities", "BldgType",
              "HouseStyle", "Exterior1st","Exterior2nd", "MasVnrType", "Foundation", 
              "BsmtFnType2", "Heating", "CentralAir", "Electrical", "FireplaceQu",
              "GarageFinish", "PavedDrive", "MiscFeature", "SaleType", "MSSubClass",
              "LotFrontage", "YearRemodAdd", "BsmtFinSF2", "BsmtUnfSF", "BsmtFullBath",
              "BsmtHalfBath","FullBath","HalfBath","TotRmsAbvGrd", "GarageYrBlt","GarageCars",
              "OpenPorchSF","EnclosedPorch","3SsnPorch","MiscVal","YrSold"]
formula2 ="SalePrice ~ "
columns2 = ""
for i in train.columns[1:-1]:
    if i not in not_sig_cols:
        if columns2 == "":
            columns2 = columns2 + "Q('" + str(i) + "')"
        else:
            columns2 = columns2 + " + Q('" + str(i) + "')"
columns2

"Q('MSZoning') + Q('LotArea') + Q('Street') + Q('LotConfig') + Q('LandSlope') + Q('Neighborhood') + Q('Condition1') + Q('Condition2') + Q('OverallQual') + Q('OverallCond') + Q('YearBuilt') + Q('RoofStyle') + Q('RoofMatl') + Q('MasVnrArea') + Q('ExterQual') + Q('ExterCond') + Q('BsmtQual') + Q('BsmtCond') + Q('BsmtExposure') + Q('BsmtFinType1') + Q('BsmtFinSF1') + Q('BsmtFinType2') + Q('TotalBsmtSF') + Q('HeatingQC') + Q('1stFlrSF') + Q('2ndFlrSF') + Q('LowQualFinSF') + Q('GrLivArea') + Q('BedroomAbvGr') + Q('KitchenAbvGr') + Q('KitchenQual') + Q('Functional') + Q('Fireplaces') + Q('GarageType') + Q('GarageArea') + Q('GarageQual') + Q('GarageCond') + Q('WoodDeckSF') + Q('ScreenPorch') + Q('PoolArea') + Q('PoolQC') + Q('Fence') + Q('MoSold') + Q('SaleCondition')"

In [14]:
final_formula_2 = formula2 + columns2
model2 = smf.ols(formula=final_formula_2, data=train).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                     106.6
Date:                Mon, 08 Aug 2022   Prob (F-statistic):               0.00
Time:                        01:01:11   Log-Likelihood:                -16670.
No. Observations:                1460   AIC:                         3.364e+04
Df Residuals:                    1311   BIC:                         3.443e+04
Df Model:                         148                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -1.526e+06   1.48e+05    -10.289      0.000   -1.82e+06   -1.23e+06
Q('MSZoning')[T.FV]            2.794e+04   1.14e+04      2.450      0.014    5566.019    5.03e+04
Q('MSZoning')[T.RH]            2.343e+04   1.15e+04      2.039      0.042     883.371     4.6e+04
Q('MSZoning')[T.RL]            2.617e+04   9719.224      2.693      0.007    7107.723    4.52e+04
Q('MSZoning')[T.RM]            1.699e+04   9159.454      1.855      0.064    -975.769     3.5e+04
Q('Street')[T.Pave]            3.533e+04   1.16e+04      3.035      0.002    1.25e+04    5.82e+04
Q('LotConfig')[T.CulDSac]      5879.5877   3055.663      1.924      0.055    -114.936    1.19e+04
Q('LotConfig')[T.FR2]         -8317.8230   3903.649     -2.131      0.033    -1.6e+04    -659.741
Q('LotConfig')[T.FR3]         -1.768e+04   1.26e+04     -1.399      0.162   -4.25e+04    7115.508
Q('LotConfig')[T.Inside]      -1845.5920   1715.340     -1.076      0.282   -5210.704    1519.520
Q('LandSlope')[T.Mod]          4688.8333   3407.698      1.376      0.169   -1996.304    1.14e+04
Q('LandSlope')[T.Sev]         -5.101e+04   1.05e+04     -4.852      0.000   -7.16e+04   -3.04e+04
Q('Neighborhood')[T.Blueste]   9557.1667   1.89e+04      0.507      0.612   -2.74e+04    4.66e+04
Q('Neighborhood')[T.BrDale]   -3595.6224   9714.224     -0.370      0.711   -2.27e+04    1.55e+04
Q('Neighborhood')[T.BrkSide]   1.272e+04   8059.845      1.579      0.115   -3089.097    2.85e+04
Q('Neighborhood')[T.ClearCr]  -4032.1336   8291.782     -0.486      0.627   -2.03e+04    1.22e+04
Q('Neighborhood')[T.CollgCr]   5876.0101   6267.095      0.938      0.349   -6418.621    1.82e+04
Q('Neighborhood')[T.Crawfor]   2.723e+04   7530.570      3.616      0.000    1.25e+04     4.2e+04
Q('Neighborhood')[T.Edwards]  -6438.1045   7039.771     -0.915      0.361   -2.02e+04    7372.344
Q('Neighborhood')[T.Gilbert]   6263.5406   6688.613      0.936      0.349   -6858.014    1.94e+04
Q('Neighborhood')[T.IDOTRR]    1.106e+04   9259.567      1.194      0.233   -7107.587    2.92e+04
Q('Neighborhood')[T.MeadowV]  -6713.3764   9423.536     -0.712      0.476   -2.52e+04    1.18e+04
Q('Neighborhood')[T.Mitchel]  -9209.0570   7254.828     -1.269      0.205   -2.34e+04    5023.284
Q('Neighborhood')[T.NAmes]     -804.8947   6785.206     -0.119      0.906   -1.41e+04    1.25e+04
Q('Neighborhood')[T.NPkVill]   2351.8132   1.02e+04      0.230      0.818   -1.77e+04    2.24e+04
Q('Neighborhood')[T.NWAmes]   -3675.9554   7087.097     -0.519      0.604   -1.76e+04    1.02e+04
Q('Neighborhood')[T.NoRidge]   4.048e+04   7413.221      5.460      0.000    2.59e+04     5.5e+04
Q('Neighborhood')[T.NridgHt]   2.469e+04   6775.901      3.644      0.000    1.14e+04     3.8e+04
Q('Neighborhood')[T.OldTown]   6487.4772   81

With fewer columns, the R-squared remained almost the same (with a reduction of only 0.01) and with those variables, the XGBoost model is going to be built.

# 3. XGBoost

To build the final modeling and with decision trees considerations the columns erased from the previous analysis are also erased for training the XGboost system alongside the columns of `Condition2`,`RoofMatl`,`GarageQual` ,and `PoolQC` as there were problems with the test columns and total columns of train and test after creating the one-hot encoding.

In [15]:
#Total columns
test_columns_2=train.columns[1:-1]
#Columns that won't be considered
not_sig_cols=["Alley", "LotShape", "LandContour", "Utilities", "BldgType",
              "HouseStyle", "Exterior1st","Exterior2nd", "MasVnrType", "Foundation", 
              "BsmtFnType2", "Heating", "CentralAir", "Electrical", "FireplaceQu",
              "GarageFinish", "PavedDrive", "MiscFeature", "SaleType", "MSSubClass",
              "LotFrontage", "YearRemodAdd", "BsmtFinSF2", "BsmtUnfSF", "BsmtFullBath",
              "BsmtHalfBath","FullBath","HalfBath","TotRmsAbvGrd", "GarageYrBlt","GarageCars",
              "OpenPorchSF","EnclosedPorch","3SsnPorch","MiscVal","YrSold","Condition2","RoofMatl",
              "GarageQual","PoolQC"]
final_cols=list(test_columns_2)
for i in not_sig_cols:
    if i in final_cols:
        final_cols.remove(i)
#Get_dummies for one-hot encoding
train_dum_2=pd.get_dummies(train[final_cols])
test_dum_2=pd.get_dummies(test[final_cols])

Then, for checking the best parameters to build the XGBRegressor a grid search was made with the parameters of `max_depth` and `n_estimators`

In [16]:
param_grid= [{'max_depth': [50,100,200],'n_estimators': [50,100,200]}]
rfr = XGBRegressor()
grid = GridSearchCV(rfr, param_grid,verbose=True,n_jobs=3).fit(train_dum_2,train["SalePrice"])


Fitting 5 folds for each of 9 candidates, totalling 45 fits


With the best parameters and with StandardScaler the final model was trained:

In [17]:
model = make_pipeline(StandardScaler(),XGBRegressor(max_depth=grid.best_params_["max_depth"],
                                                    n_estimators=grid.best_params_["n_estimators"]
                                                    ,n_jobs=-1),verbose=True).fit(train_dum_2,train["SalePrice"])

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ...... (step 2 of 2) Processing xgbregressor, total=   8.1s


Finally, the prices are predicted and saved on the `submission.csv` file:

In [18]:
y_fin=list(model.predict(test_dum_2))
#Sample
y_fin[:5]

[121664.79, 158219.62, 190531.89, 189584.73, 195687.06]

In [19]:
test["SalePrice"]=y_fin
#Output
Final_prediction=test[["Id","SalePrice"]]

In [20]:
Final_prediction

,Id,SalePrice
0,1461,121664.789062
1,1462,158219.625000
2,1463,190531.890625
3,1464,189584.734375
4,1465,195687.062500
...,...,...
1454,2915,86019.335938
1455,2916,80808.617188
1456,2917,167423.015625
1457,2918,112016.023438


In [21]:
Final_prediction.to_csv("submission.csv",index=False)